# 基于MindSpore构造卷积层

本实验主要专注于卷积层的原理和构造，使用MindSpore构造卷积层。

## 1、实验目的

- 掌握卷积层的原理。
- 掌握如何使用MindSpore构造卷积层。

## 2、卷积层原理介绍

卷积层的主要作用是提取图像中的特征，浅层卷积层用来提取边缘、纹理等特征，深层卷积层用来提取更加抽象的特征，整合在一起构成了丰富的图像特征。

卷积层由一组和输入层有着相同通道数的卷积核构成，将卷积核在输入层上按照指定步长滑动，每到一个位置计算卷积核权重和对应输入特征的乘积，所有乘积求和作为输出特征图的一个特征值，单个卷积核在输入特征上滑动一遍就得到了单通道的输出特征，一组卷积核全部进行上述过程就得到了包含若干通道的输出特征图。这种每次滑动到一个局部位置的计算方式能够在非常大的程度上减少参数量，同时一个物体无论在图像中的什么位置，这一组卷积核都能够提取其特征，具有平移不变性。

以单通道为例，卷积的简易过程如图所示，从左到右三部分依次为输入图像、卷积核（过滤器）、输出特征，输入图像尺寸为5×5，卷积核尺寸为3×3，输出特征图尺寸为3×3。卷积核中心滑动到输入图像中某一像素的位置，则重叠的部分分别计算乘积，最后求和作为当前位置的输出特征值。
<img src="./Figures/fig001.jpg" style="zoom:70%" />

图中的卷积计算过程如下：

第一行卷积的结果为：1×(-1) + 0×(-1) + 0×(-1) = -1 

第二行卷积的结果为：1×(-1) + 0× 1 + 1×(-1) = -2

第三行卷积的结果为：1×(-1) + 0×(-1) + 1×(-1) = -2

然后把三行的结果相加等于-5。

卷积层输出特征图尺寸计算方式：输入特征图尺寸为$w_i×h_i×c_i$（分别为宽、高、通道数），卷积核的尺寸为$k_i×k_i$（k表示卷积核的边长），输出通道数为$n_{i+1}$，卷积步长为$s$，输入特征图周围填充0特征值的行数为$p_0$，输出特征图的尺寸为$w_{i+1}×h_{i+1}×c_{i+1}$，则输出特征图的尺寸为：
$$w_{i+1}=\frac{w_i-k_i+2p_0}{s}+1$$
$$h_{i+1}=\frac{h_i-k_i+2p_0}{s}+1$$
$$c_{i+1}=n_{i+1}$$

当卷积结束之后，可以发现图像变小了，原来是5×5的，现在变成了3×3的，这不是想要的结果，因此进行零填充，因为填充的是二维图像，所以在图像四周都要填成0。填充完之后再开始做卷积运算，中间部分已得出，需要计算新填充的部分，用同样的方式计算完所有的位置，二维卷积的零填充如图所示：
<img src="./Figures/fig002.jpg" style="zoom:70%" />
原来的图像是5×5的，零填充之后，得到的也是5×5，大小保持不变。
根据上面的公式，我们可以计算输出特征图的尺寸。输入特征图尺寸为5×5×1，卷积核的尺寸为3×3，卷积步长$s$为1，输入特征图周围填充0特征值的行数$p_0$为1，根据公式得到输出特征图的尺寸为5×5×1。
$$w_{i+1}=\frac{5-3+2}{1}+1=5$$
$$h_{i+1}=\frac{5-3+2}{1}+1=5$$
$$c_{i+1}=n_{i+1}=1$$

## 3、实验环境

在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

## 4、数据处理

### 4.1 数据准备

对输入Tensor计算二维卷积。该Tensor的常见shape为$(N,C_{in},H_{in},W_{in})$，其中N为batch size，$C_{in}$为空间维度，$H_{in}$,$W_{in}$分别为特征层的高度和宽度。现给出Tensor的shape为(1,120,1024,640)计算二维卷积。

### 4.2 数据加载

导入需要的Python库，调用mindspore.nn.Conv2d()，它是对输入Tensor计算二维卷积，其中has_bias设置为默认值'False'，weight_init设置为'normal'，然后用给出的Tensor进行数据加载，Tensor的shape为(1,120,1024,640)。

In [8]:
from mindspore import Tensor
import mindspore.nn as nn
import numpy as np
from mindspore import dtype as mstype

# 给出Tensor的shape为(1,120,1024,640)计算二维卷积。
net = nn.Conv2d(120, 240, 4, has_bias=False, weight_init='normal')
# 输入
x = Tensor(np.ones([1, 120, 1024, 640]), mstype.float32)

## 5、模型构建

**导入Python库&模块**

在使用前，导入需要的Python库。

In [9]:
# 导入科学计算库
import numpy as np
# 导入神经网络模块
import mindspore.nn as nn
# 被初始化的Tensor的数据类型
from mindspore import dtype as mstype
# 常见操作
from mindspore.nn import Conv2d
# 用于初始化Tensor的Tensor
from mindspore import Tensor
# MindSpore中神经网络的基本构成单元
from mindspore.nn import Cell

## 6、模型预测

给定Tensor的shape为(1,120,1024,640)，然后调用Conv2d函数来计算二维卷积，最后输出结果。

In [10]:
net = Conv2d(120, 240, 3, has_bias=False, weight_init='normal')
# 输入
x = Tensor(np.ones([1, 120, 1024, 640]), mstype.float32)
# 输出
output = net(x)
print(output[:,6])
print(output.shape)

[[[-0.2439619  -0.35941112 -0.35941112 ... -0.35941112 -0.35941112
   -0.36657017]
  [-0.28095603 -0.35479867 -0.35479867 ... -0.35479867 -0.35479867
   -0.39644748]
  [-0.28095603 -0.35479867 -0.35479867 ... -0.35479867 -0.35479867
   -0.39644748]
  ...
  [-0.28095603 -0.35479867 -0.35479867 ... -0.35479873 -0.35479873
   -0.39644748]
  [-0.28095603 -0.35479867 -0.35479867 ... -0.35479867 -0.35479867
   -0.39644748]
  [-0.00274147 -0.16676085 -0.16676085 ... -0.16676083 -0.16676083
   -0.33913577]]]
(1, 240, 1024, 640)
